In [ ]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
using Printf
using JSON
using CSV
using DataFrames
using ColorSchemes
using LaTeXStrings

In [ ]:
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dynamics_parameters()
paramMulti = SailorMoon.multi_shoot_parameters(param3b)
gr()
fs = 16


In [ ]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

In [ ]:
moon     = SailorMoon.plot_circle((1-param3b.mu2)*param3b.lstar, 0.0, 0.0)
earth    = SailorMoon.plot_circle(param3b.mu2*param3b.lstar, 0.0, 0.0)
earth_ub = SailorMoon.plot_circle(param3b.mu2*param3b.lstar+6375,0.0, 0.0)
earth_lb = SailorMoon.plot_circle(param3b.mu2*param3b.lstar-6375, 0.0, 0.0)
moon_soi_outer = plot_circle((1-param3b.mu2)*param3b.lstar+66000, 0.0, 0.0);

In [ ]:
function interpolatecolor(cs,value, vmin, vmax)
    cs = cgrad(cs)
    color = cs[(value - vmin) / (vmax - vmin)]
    return color
end

In [ ]:
### PARAMETERS ###################################
# csv file to load the initial solution
filename = "../run/data/diffcorr_0619_NoThrust.csv"
# dv_dir function corresponding to the csv file 
dir_func = SailorMoon.dv_no_thrust

n_arc = 5

cs = :lightrainbow 
##################################################

if dir_func == SailorMoon.dv_no_thrust
    τ_ig = 0.0
else 
    τ_ig = 1.0
end

# load initial guess
df = CSV.read(filename, DataFrame; header=0);

arcs_color = Dict(
    "xlr_bck_arc" => :blue, 
    "xlr_fwd_arc" => :red,
    "mid_bck_arc" => :green, 
    "mid_fwd_arc" => :black, 
    "lpo_arc" => :yellow
)






In [ ]:
p_ = plot(
    size=(700,500), 
    frame_style=:box, aspect_ratio=:equal, grid=0.4, 
    legend=false, 
    xlabel=L"t", ylabel=L"val",
    colorbar_titlefontsize=fs,
    fontsize=fs, xlabelfontsize=fs, ylabelfontsize=fs, xtickfontsize=fs-1, ytickfontsize=fs-1, legendfontsize=fs-1
);


tofs= [row[1] for row in eachrow( df ) ] 
tof_max = maximum(tofs)
tof_min = minimum(tofs)

tof_max_days = tof_max * param3b.tstar / (24*60*60)
tof_min_days = tof_min * param3b.tstar / (24*60*60)

colors = [interpolatecolor(cs, row[1], tof_min, tof_max) for row in eachrow(df)]

u = []
t = []

# for (i, row) in enumerate( eachrow( df ) ) 
    
row = df[1,:]

    x = collect(values(row[4:end]))
#     println(x)
    
#     color = colors[i]
    res, sol_param_list, sols_ballistic, tofs = SailorMoon.multishoot_trajectory2(x, dir_func, paramMulti, true, false)
        
    # ballistic legs
    for sol_ballistic in sols_ballistic

        u = vcat(sol_ballistic.u[:])
        t = vcat(sol_ballistic.t[:])
        
    end
    
    for j = 1:Int(floor(length(sol_param_list)/paramMulti.n_arc))
        
        for k = 1:paramMulti.n_arc
                        
            if mod(j,2) == 1
                # backward propagation
                sol, _, name = sol_param_list[length(sol_param_list) - j*paramMulti.n_arc + k]
                u = vcat(u, sol.u[:])
                t_append = sol.t[:] .+ t[end]
                t = vcat(t, t_append)
                
            else
                # forward propagation
                sol, _, name = sol_param_list[length(sol_param_list)-(j-1)*paramMulti.n_arc - k + 1]
                u_append = sol.u[end:-1:1, :]
                t_append = sol.t[end:-1:1] .- sol.t[end] .+ t[end]
                u = vcat(u, u_append)
                t = vcat(t, t_append)
            end

        end
        
    end
    
# end
u = Array(u)
u = hcat(u...)[:,:]


In [ ]:
scatter!(p_, t[:], u[1,:])


In [ ]:
scatter(collect(1:1368),t)

In [ ]:
## test 
t_, u_  = SailorMoon.x2time_series(x, dir_func, paramMulti)


In [ ]:
plot(t_, u_[1,:])

In [ ]:
# compute JC from state in EMrot
function compute_JC(state::Vector)
    x, y, z = state[1], state[2], state[3]
    vx, vy, vz = state[4], state[5], state[6]
    r13 = norm([x-param3b.mu2, y, z])  # Earth - SC
    r23 = norm([x-param3b.mu1, y, z])  # Moon - SC
    
    U = 1/2*(x^2+y^2) + param3b.mu1 / r13 + param3b.mu2 / r23 
    JC = 2*U - (vx^2 + vy^2 + vz^2)
    
    return JC 
end

In [ ]:
θlpo = x[19+12*paramMulti.n_arc]

In [ ]:
u_emrots = []
JC_vec = []

for (i, tnow) in enumerate(t_)
    θm = θlpo - param3b.oml * tnow
    u_sb1   = u_[1:6,i]
    u_emrot = SailorMoon.transform_SunB1_to_EMrot(u_sb1, θm, param3b.oml, param3b.as)
    u_emrots = push!(u_emrots, transpose(u_emrot))
    
    jc = compute_JC(u_emrot)
    push!(JC_vec, jc)
end
u_emrots = Array(reshape(hcat(u_emrots...), 6, size(u_emrots,1)))



In [ ]:
plot(t_, JC_vec, ylim=[0,4], xlabel=L"t,s", ylabel=L"JC")

In [ ]:
u_emrots = Array(reshape(hcat(u_emrots...), 6, size(u_emrots,1)))
plot(u_emrots[1,:], u_emrots[2,:])
